In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
import jax.numpy as jnp
import jax
import jax.random as jr
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

is_dark = False
theme, cs = rp.mpl_setup(is_dark)
rp.plotly_setup(is_dark)

In [2]:
%cd ~/cdv/

/home/nmiklaucic/cdv


/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from pathlib import Path
import pyrallis
from cdv.config import MainConfig
import orbax.checkpoint as ocp

from cdv.training_state import TrainingRun
from cdv.checkpointing import best_ckpt

with open('configs/test.toml') as conf_file:
    config = pyrallis.cfgparsing.load(MainConfig, conf_file)

model = config.build_regressor()
model


MaceModel(
    # attributes
    num_species = 89
    elem_indices = Array([  -1,   65,   87,    6,   37,   59,   75,   82,   84,   85,   88,
              4,   28,   39,   51,   64,   77,   83,   86,    1,    7,   31,
             35,   41,   43,   36,   47,   48,   53,   49,   42,   46,   57,
             63,   76,   80,   81,    2,    5,   20,   29,   38,   62,   52,
             69,   71,   68,   54,   44,   45,   55,   60,   61,   79,   78,
              0,    3,    9,   12,   14,   15,   13,   16,   17,   18,   10,
             19,   21,   22,   23,   11,   24,   26,   34,   73,   50,   67,
             66,   70,   74,   56,   40,   72,   58, 1000, 1000, 1000, 1000,
           1000,    8,   27,   33,   32,   30,   25], dtype=int32)
    output_graph_irreps = '1x0e'
    output_node_irreps = None
    hidden_irreps = '128x0e+64x1o+32x2e'
    readout_mlp_irreps = '128x0e+64x1o+32x2e'
    scalar_mean = -6.193478848755548
    scalar_std = -6.193478848755548
    num_interactions = 3
    

In [4]:
from cdv.dataset import load_file, stack_trees
cg = load_file(config)
cg

CrystalGraphs(nodes=(1024, 3), edges=(1024, 16), graphs=(32,))

In [5]:
from cdv.layers import Context
ctx = Context(training=True)
out, params = model.init_with_output(jr.key(0), cg, ctx=ctx)
out


Array([[-5.53125],
       [-5.84375],
       [-4.71875],
       [-6.3125],
       [-6.1875],
       [-6.09375],
       [-6.09375],
       [-6.375],
       [-6.09375],
       [-6.15625],
       [-6.1875],
       [-6.4375],
       [-5.8125],
       [-6.28125],
       [-5.9375],
       [-5.6875],
       [-6.28125],
       [-6.1875],
       [-5.0625],
       [-5.65625],
       [-6.8125],
       [-5.65625],
       [-6.1875],
       [-6.1875],
       [-5.84375],
       [-6.125],
       [-4.8125],
       [-4.21875],
       [-6.21875],
       [-5.625],
       [-6.625],
       [-6.1875]], dtype=bfloat16)

In [6]:
cg.nodes


NodeData(
    species=Array([6, 6, 6, ..., 0, 0, 0], dtype=int16),
    cart=Array([[ -2.5456254, -13.569227 ,  -5.146564 ],
       [-10.481925 ,  -7.1994514,  -0.9888134],
       [-13.326975 ,  -6.646334 ,  -7.1241913],
       ...,
       [  0.       ,   0.       ,   0.       ],
       [  0.       ,   0.       ,   0.       ],
       [  0.       ,   0.       ,   0.       ]], dtype=float32),
    graph_i=Array([ 0,  0,  0, ..., 31, 31, 31], dtype=int32)
)

In [7]:
from cdv.regression import EFSWrapper


out = EFSWrapper()(model, params, cg=cg, ctx=ctx)
out

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/jax/_src/api.py:763 in        │
│ _check_scalar                                                                                    │
│                                                                                                  │
│    760 def _check_scalar(x):                                                                     │
│    761   msg = "Gradient only defined for scalar-output functions. Output {}.".format            │
│    762   try:                                                                                    │
│ ❱  763 │   aval = core.get_aval(x)                                                               │
│    764   except TypeError as e:                                                                  │
│    765 │   raise TypeError(msg(f"was {x}")) from e                                               │
│    766   else:                                                                                   │
│                                                                                                  │
│ /home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/jax/_src/core.py:1480 in      │
│ get_aval                                                                                         │
│                                                                                                  │
│   1477   if isinstance(x, Tracer):                                                               │
│   1478 │   return x.aval                                                                         │
│   1479   else:                                                                                   │
│ ❱ 1480 │   return concrete_aval(x)                                                               │
│   1481                                                                                           │
│   1482                                                                                           │
│   1483 def concretization_function_error(fun, suggest_astype=False):                             │
│                                                                                                  │
│ /home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/jax/_src/core.py:1472 in      │
│ concrete_aval                                                                                    │
│                                                                                                  │
│   1469 │   if handler: return handler(x)                                                         │
│   1470   if hasattr(x, '__jax_array__'):                                                         │
│   1471 │   return concrete_aval(x.__jax_array__())                                               │
│ ❱ 1472   raise TypeError(f"Value {x!r} with type {type(x)} is not a valid JAX "                  │
│   1473 │   │   │   │      "type")                                                                │
│   1474                                                                                           │
│   1475                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Value (Array([-171, -181, -146, -196, -192, -189, -189, -198, -189, -191, -192,
       -200, -180, -195, -184, -176, -195, -192, -158, -175, -211, -175,
       -192, -192, -181, -190, -149, -131, -193, -174, -205, -192],      dtype=bfloat16), Array([[-5.53125],
       [-5.84375],
       [-4.71875],
       [-6.3125],
       [-6.1875],
       [-6.09375],
       [-6.09375],
       [-6.375],
       [-6.09375],
       [-6.15625],
       [-6.1875],
       [-6.4375],
       [-5.8125],
       [-6.28125],
       [-5.9375],
       [-5.6875],
       [-6.28125]

In [ ]:
loss = config.train.loss.efs_loss(cg, out)

In [ ]:
cg.e_form.shape